# 07 DENSE-SPLIT-REG

In [ ]:
import config
from utils import data_model
from utils.model import Model

import pandas as pd
import numpy as np
import tensorflow as tf

## Load data

In [ ]:
data, imgs_left, imgs_right = data_model.load(
    config.PATH_DATA_FEATURES01_DLIB_AUGMENTED_NORM_CSV,
    config.PATH_DATA_FEATURES01_DLIB_AUGMENTED_NORM_IMGS_LEFT,
    config.PATH_DATA_FEATURES01_DLIB_AUGMENTED_NORM_IMGS_RIGHT
)

In [ ]:
data.head()

## Split data

In [ ]:
(
    (train_data, train_imgs_left, train_imgs_right),
    (validation_data, validation_imgs_left, validation_imgs_right),
    (test_data, test_imgs_left, test_imgs_right)
) = data_model.split(
    data, imgs_left, imgs_right,
    train_size=0.95,
    validation_size=0.95,
    random_state=42
)

In [ ]:
print("Train length: {}".format(len(train_data)))
print("Validation length: {}".format(len(validation_data)))
print("Test length: {}".format(len(test_data)))

## Model

### Architecture

![](img/models/07-DSR.png)

In [ ]:
def get_model(features, left_imgs, right_imgs, keep_prob):
    with tf.variable_scope('model'):
        # Left Eye
        l_flat = tf.contrib.layers.flatten (left_imgs)
        l_d1 = tf.layers.dense(l_flat, 256, activation=tf.nn.relu)
        l_d2 = tf.layers.dense(l_d1, 64, activation=tf.nn.relu)
        l_drop = tf.nn.dropout(x=l_d2, keep_prob=keep_prob)
        
        # Right Eye
        r_flat = tf.contrib.layers.flatten (right_imgs)
        r_d1 = tf.layers.dense(r_flat, 256, activation=tf.nn.relu)
        r_d2 = tf.layers.dense(r_d1, 64, activation=tf.nn.relu)
        r_drop = tf.nn.dropout(x=r_d2, keep_prob=keep_prob)
        
        # Combine all
        all_concat = tf.concat(
            values=[features, l_drop, r_drop],
            axis=1
        )
        all_d1 = tf.layers.dense(all_concat, 256, activation=tf.nn.relu)
        all_drop = tf.nn.dropout(x=all_d1, keep_prob=keep_prob)
        all_d2 = tf.layers.dense(all_drop, 64, activation=tf.nn.relu)
        all_d3 = tf.layers.dense(all_concat, 2, activation=None)
        
    return all_d3

In [ ]:
print("Parameters to adjust: {}".format( 
    (
        (20*30*256)+
        256*64
    )*2+
    (64*2+12)*256+
    256*64+
    64*2
))

### Parameters

In [ ]:
MODEL_NAME = 'DSR-07'

EPOCHS = 100
BATCH_SIZE = 256
LEARNING_RATE = 0.0005
KEEP_PROB = 0.85

## Runs

In [ ]:
model = Model(MODEL_NAME, get_model)

### Train

In [ ]:
model.train(
    train_data, train_imgs_left, train_imgs_right,
    validation_data, validation_imgs_left, validation_imgs_right,
    BATCH_SIZE, EPOCHS, LEARNING_RATE, KEEP_PROB
)

### Test

In [ ]:
model_test = Model(MODEL_NAME, saved_model=MODEL_NAME+".final")

In [ ]:
model_test.test(
    test_data, test_imgs_left, test_imgs_right
)

### Results


| Name | Epochs | Batch Size | Learning rate | Keep_prob | Train | Validation | Test |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| DSR-01 | 25 | 256 | 0.001 | 1.0 | 0.09392019361257553 | 0.16234776377677917 | 0.16432613 |
| DSR-02 | 25 | 256 | 0.001 | 0.9 | 0.10075189918279648 | 0.16525883972644806 | 0.16685201 |
| DSR-03 | 25 | 256 | 0.001 | 0.7 | 0.12321840226650238 | 0.17533698678016663 | 0.17595138 |
| DSR-04 | 50 | 256 | 0.0005 | 0.85 | 0.08604488521814346 | 0.16130870580673218 | 0.16228983 |
| DSR-05 | 50 | 512 | 0.0005 | 0.85 | 0.08791523426771164 | 0.1647828370332718 | 0.16689432 |
| DSR-06 | 100 | 256 | 0.0001 | 0.9 | 0.0811753049492836 | 0.16763710975646973 | 0.16976333 |
| DSR-07 | 100 | 256 | 0.0005 | 0.85 | 0.06679268181324005 | 0.1450866460800171 | 0.14738837 |